In [1]:
import torch
import torchaudio
import os
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torchaudio import transforms
from torch.nn import Sequential
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset

In [2]:
import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# sample_rate=16000
# n_mfcc = 40
# n_fft = 512
# n_mels = 256
# hop_length = 10

# mfcc_transform = nn.Sequential(
#     torchaudio.transforms.MFCC(
#         sample_rate=sample_rate,
#         n_mfcc=n_mfcc,
#         melkwargs={
#           'n_fft': n_fft,
#           'n_mels': n_mels,
#           'hop_length': hop_length,
#           'mel_scale': 'htk',
#         }
#     )
# )

In [3]:
# dic = {}
# i = 0
# def data_preprocessing(data):
#     mfccs = []
#     labels = []
#     for (waveform, sample_rate, utterance, speaker_id, chapter_id, utterance_id) in data:
#         mfcc = mfcc_transform(waveform).squeeze(0).transpose(0, 1).to(device)
#         rand = random.randrange(0, len(mfcc) - 300)
#         mfccs.append(mfcc[rand : rand + 300,:])
#         if speaker_id in dic:
#             labels.append(dic.get(speaker_id))
#         else:
#             global i
#             dic[speaker_id] = i
#             labels.append(i)
#             i=i+1

#     mfccs = torch.nn.utils.rnn.pad_sequence(mfccs, batch_first=True).transpose(1, 2)
#     labels = torch.Tensor(labels)
    
#     return mfccs, labels

In [4]:
batch_size_train = 64
batch_size_test = 64

mfccs = torch.load('train_mfccs_251')
labels = torch.load('train_labels_251')
train_data = torch.utils.data.TensorDataset(mfccs, labels)
mfccs = torch.load('test_mfccs_251')
labels = torch.load('test_labels_251')
test_data = torch.utils.data.TensorDataset(mfccs, labels)

train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size=batch_size_train,
                                           shuffle=True,
                                           drop_last=True
                                           )
test_loader = torch.utils.data.DataLoader(test_data,
                                          batch_size=batch_size_test,
                                          shuffle=True,
                                          drop_last=True
                                         )
print('train:', len(train_data), 'test:', len(test_data))

FileNotFoundError: [Errno 2] No such file or directory: 'train_mfccs_251'

In [ ]:
def plot():    
    fig = plt.figure()
    counter = [i for i in range(1, itera_test+1)]
    plt.plot(counter, test_losses, color='red')
    plt.plot(counter, avg_train_loss, color='black')
    plt.legend(['Test Loss', 'Average Train Loss'], loc='upper right')
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.show()

In [ ]:
def plotacc():
    fig = plt.figure()
    acc_counter = [i for i in range(1, itera_test+1)]
    plt.plot(acc_counter, acc, color='red')
    plt.legend(['Accuracy'], loc='upper right')
    plt.xlabel('epochs')
    plt.ylabel('accuracy')
    plt.show()

In [ ]:
class xvecTDNN(nn.Module):

    def __init__(self, numSpkrs, p_dropout, momentum):
        super(xvecTDNN, self).__init__()
        
        self.tdnn1 = nn.Conv1d(in_channels=40, out_channels=512, kernel_size=5, dilation=1)
        self.bn_tdnn1 = nn.BatchNorm1d(512, momentum=momentum, affine=False)
        self.dropout_tdnn1 = nn.Dropout(p=p_dropout)

        self.tdnn2 = nn.Conv1d(in_channels=512, out_channels=512, kernel_size=5, dilation=2)
        self.bn_tdnn2 = nn.BatchNorm1d(512, momentum=momentum, affine=False)
        self.dropout_tdnn2 = nn.Dropout(p=p_dropout)

        self.tdnn3 = nn.Conv1d(in_channels=512, out_channels=512, kernel_size=7, dilation=3)
        self.bn_tdnn3 = nn.BatchNorm1d(512, momentum=momentum, affine=False)
        self.dropout_tdnn3 = nn.Dropout(p=p_dropout)

        self.tdnn4 = nn.Conv1d(in_channels=512, out_channels=512, kernel_size=1, dilation=1)
        self.bn_tdnn4 = nn.BatchNorm1d(512, momentum=momentum, affine=False)
        self.dropout_tdnn4 = nn.Dropout(p=p_dropout)

        self.tdnn5 = nn.Conv1d(in_channels=512, out_channels=1500, kernel_size=1, dilation=1)
        self.bn_tdnn5 = nn.BatchNorm1d(1500, momentum=momentum, affine=False)
        self.dropout_tdnn5 = nn.Dropout(p=p_dropout)

        self.fc1 = nn.Linear(3000, 512)
        self.bn_fc1 = nn.BatchNorm1d(512, momentum=momentum, affine=False)
        self.dropout_fc1 = nn.Dropout(p=p_dropout)

        self.fc2 = nn.Linear(512, 512)
        self.bn_fc2 = nn.BatchNorm1d(512, momentum=momentum, affine=False)
        self.dropout_fc2 = nn.Dropout(p=p_dropout)

        self.fc3 = nn.Linear(512, numSpkrs)

    def forward(self, x):
        x = self.dropout_tdnn1(F.relu(self.bn_tdnn1(self.tdnn1(x))))
        x = self.dropout_tdnn2(F.relu(self.bn_tdnn2(self.tdnn2(x))))
        x = self.dropout_tdnn3(F.relu(self.bn_tdnn3(self.tdnn3(x))))
        x = self.dropout_tdnn4(F.relu(self.bn_tdnn4(self.tdnn4(x))))
        x = self.dropout_tdnn5(F.relu(self.bn_tdnn5(self.tdnn5(x))))
        
        stats = torch.cat((x.mean(dim=2), x.std(dim=2)), dim=1)
        x = self.dropout_fc1(F.relu(self.bn_fc1(self.fc1(stats))))
        x = self.dropout_fc2(F.relu(self.bn_fc2(self.fc2(x))))
        x = self.fc3(x)
        
        return x


In [ ]:
learning_rate = 1e-4
log_interval = int(2280 / batch_size_train)
model = xvecTDNN(numSpkrs=251, p_dropout=0, momentum=0.1).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.0001)
# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 15], gamma=0.1)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)
train_losses = []
train_counter = []
avg_train_loss = []
avg_train_counter = []
itera = 1
def train():
    global itera
    model.train()
    running_loss = 0
    train_loss = 0
    for batch_idx, data in enumerate(train_loader):
        mfccs, labels = data 
        mfccs, labels = mfccs.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(mfccs)
        loss = criterion(output, labels.long())
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        train_loss += loss.item() / len(train_loader)
#         train_losses.append(loss.item())
#         train_counter.append((batch_idx*batch_size_train) + ((itera-1)*len(train_loader.dataset)))
        # 每隔10个batch输出，batch_size=8,所以每隔80个数据输出一次
        if batch_idx != 0 and batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.4f}'.format(
                epoch,
                batch_idx * len(mfccs), 
                len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                running_loss / log_interval))
                # sum(train_losses) / batch_idx))
                # loss.item()))
            train_losses.append(running_loss / log_interval)
            train_counter.append((batch_idx*batch_size_train) + ((itera-1)*len(train_loader.dataset)))
            running_loss = 0
    print('\nTrain set: Avg. loss: {:.4f}\n'.format(train_loss))
    avg_train_loss.append(train_loss)
    avg_train_counter.append(itera*len(train_loader.dataset))
    scheduler.step()         
    itera += 1
    
    return model

In [ ]:
test_losses = []
test_counter = []
acc = []
itera_test = 1

def test():
    global itera_test
    model.eval()
    test_loss = 0
    correct = 0
    class_in=[] #定义类内相似度列表
    class_each=[] #定义类间相似度列表
    with torch.no_grad():
        for batch_idx, data in enumerate(test_loader):
            mfccs, labels = data 
            mfccs, labels = mfccs.to(device), labels.to(device)
            output = model(mfccs)
            loss = criterion(output, labels.long())
            test_loss += loss.item() / len(test_loader)
            # output.shape: [batchsize, 251]
            
            # get the index of the max log-probability in rows 
            # return the corresponding label
            value, pred = torch.max(output.data, dim=1)

            print(value)

            # pred.shape && labels.shape: [batchsize]
            # .eq逐个元素判断是否相等 相等1 不相等0
            correct += pred.eq(labels).sum()
            
            # 需要识别的用户id和模型id一样，就认为是类内测试
            # 否则是类间测试
            # 对象是概率值
            for i in range(batch_size_test):
                if pred[i] == labels[i]:
                    class_in.append(value[i].cpu().numpy())
                else:
                    class_each.append(value[i].cpu().numpy())
    
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss,
        correct,
        len(test_loader.dataset),
        100. * correct / len(test_loader.dataset))) 

    test_losses.append(test_loss)
    test_counter.append(len(train_loader.dataset)*itera_test)
    acc.append(correct.detach().cpu().numpy() / len(test_loader.dataset))
    
    plot()
    plotacc()
    itera_test += 1

In [ ]:
epochs = 30
for epoch in range(1, epochs + 1):
    train()
    test()
torch.save(model.state_dict(), 'model.pth')
print('Finished training and testing')

In [ ]:
for key, value in dic.items():
    print('{key}:{value}'.format(key = key, value = value))